In [1]:
!pip install wikipedia==1.4.0
!pip install scikit-learn==1.0.2
!pip install gensim==4.0.1

Keyring is skipped due to an exception: 'keyring.backends'


Keyring is skipped due to an exception: 'keyring.backends'


Keyring is skipped due to an exception: 'keyring.backends'


In [4]:
import wikipedia as wiki

k = 4
question = "What is global warming?"

results = wiki.search(question, results=k)
print('Question:', question)
print('Answers: ', results)

Question: What is global warming?
Answers:  ['Climate change', 'Global warming (disambiguation)', 'Effects of climate change', 'Climate change denial']


# Data Exploration

In this section, we are going to load a json file into a pandas.DataFrame. At last, elaborate our list of documents.

In [5]:
import json
import numpy as np
import pandas as pd

In [6]:
import os

# list the available data
for dirname, _, filenames in os.walk('D:/Semester 2/ENFUSE/pdf_data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/Semester 2/ENFUSE/pdf_data/dev-v1.1.json
D:/Semester 2/ENFUSE/pdf_data/train-v1.1.json


In [7]:
def squad_json_to_dataframe(file_path, record_path=['data','paragraphs','qas','answers']):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    """
    file = json.loads(open(file_path).read())
    # parsing different level's in the json file
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file,record_path[:-2])
    # combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    data = m[['id','question','context','answers']].set_index('id').reset_index()
    data['c_id'] = data['context'].factorize()[0]
    return data

In [8]:
# loading the data
file_path = 'D:/Semester 2/ENFUSE/pdf_data/train-v1.1.json'
data = squad_json_to_dataframe(file_path)
data

,id,question,context,answers,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 515, 'text': 'Saint Bernadet...",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 188, 'text': 'a copper statu...",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 279, 'text': 'the Main Build...",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 381, 'text': 'a Marian place...",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 92, 'text': 'a golden statue...",0
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 229, 'text': 'Oregon'}]",18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 414, 'text': 'Rangoon'}]",18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 476, 'text': 'Minsk'}]",18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 199, 'text': '1975'}]",18890


In [9]:
# how many documents do we have?
data['c_id'].unique().size

18891

# Get the Unique Documents

Let's select the unique documents in our data. This will be the list of documents to search for the answers.

In [10]:
documents = data[['context', 'c_id']].drop_duplicates().reset_index(drop=True)
documents

,context,c_id
0,"Architecturally, the school has a Catholic cha...",0
1,"As at most other universities, Notre Dame's st...",1
2,The university is the major seat of the Congre...,2
3,The College of Engineering was established in ...,3
4,All of Notre Dame's undergraduate students are...,4
...,...,...
18886,"Institute of Medicine, the central college of ...",18886
18887,Football and Cricket are the most popular spor...,18887
18888,The total length of roads in Nepal is recorded...,18888
18889,The main international airport serving Kathman...,18889


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# defining the TF-IDF
tfidf_configs = {
    'lowercase': True,
    'analyzer': 'word',
    'stop_words': 'english',
    'binary': True,
    'max_df': 0.9,
    'max_features': 10_000
}
# defining the number of documents to retrieve
retriever_configs = {
    'n_neighbors': 10,
    'metric': 'cosine'
}

# defining our pipeline
embedding = TfidfVectorizer(**tfidf_configs)
retriever = NearestNeighbors(**retriever_configs)

In [12]:
# let's train the model to retrieve the document id 'c_id'
X = embedding.fit_transform(documents['context'])
retriever.fit(X, documents['c_id'])

NearestNeighbors(metric='cosine', n_neighbors=10)

Let's test the vectorizer, what information our model is using to extract the vector?

In [13]:
def transform_text(vectorizer, text):
    '''
    Print the text and the vector[TF-IDF]
    vectorizer: sklearn.vectorizer
    text: str
    '''
    print('Text:', text)
    vector = vectorizer.transform([text])
    vector = vectorizer.inverse_transform(vector)
    print('Vect:', vector)

In [14]:
# vectorize the question
transform_text(embedding, question)

Text: What is global warming?
Vect: [array(['warming', 'global'], dtype='<U18')]


What is the most similar document to this question?

In [15]:
# predict the most similar document
X = embedding.transform([question])
c_id = retriever.kneighbors(X, return_distance=False)[0][0]
selected = documents.iloc[c_id]['context']

# vectorize the document
transform_text(embedding, selected)

Text: In glaciated areas where the glacier moves faster than one km per year, glacial earthquakes occur. These are large scale temblors that have seismic magnitudes as high as 6.1. The number of glacial earthquakes in Greenland peaks every year in July, August and September and is increasing over time. In a study using data from January 1993 through October 2005, more events were detected every year since 2002, and twice as many events were recorded in 2005 as there were in any other year. This increase in the numbers of glacial earthquakes in Greenland may be a response to global warming.
Vect: [array(['year', 'warming', 'using', 'twice', 'time', 'study', 'september',
       'scale', 'response', 'recorded', 'peaks', 'october', 'occur',
       'numbers', 'number', 'moves', 'large', 'km', 'july', 'january',
       'increasing', 'increase', 'high', 'greenland', 'global', 'glacier',
       'glacial', 'faster', 'events', 'earthquakes', 'detected', 'data',
       'august', 'areas', '2005', 

# Evaluation

In [16]:
%%time
# predict one document for each question
X = embedding.transform(data['question'])
y_test = data['c_id']
y_pred = retriever.kneighbors(X, return_distance=False)

Wall time: 37 s


In [17]:
# top documents predicted for each question
y_pred

array([[    0,  3694, 10613, ..., 17590,  6913,  6912],
       [    7,  1469,     2, ...,    29, 14201,    17],
       [   38,  1469, 14152, ...,    28,     7, 14201],
       ...,
       [18890, 18884, 18836, ..., 12302, 18837,  4200],
       [18890,  3537, 18841, ..., 16014, 18884, 10882],
       [12592, 12591, 12598, ..., 12593, 12600, 12588]], dtype=int64)

In [18]:
def top_accuracy(y_true, y_pred) -> float:
    right, count = 0, 0
    for i, y_t in enumerate(y_true):
        count += 1
        if y_t in y_pred[i]:
            right += 1
    return right / count if count > 0 else 0

In [19]:
acc = top_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')
print('Quantity:', int(acc*len(y_pred)), 'from', len(y_pred))

Accuracy: 0.7148
Quantity: 62615 from 87599


In [20]:
from gensim.parsing.preprocessing import preprocess_string

# create a corpus of tokens
corpus = documents['context'].tolist()
corpus = [preprocess_string(t) for t in corpus]

C:\Users\Sudharsan Ramesh\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [21]:
from gensim.models import Word2Vec
import gensim.downloader

# you can download a pretrained Word2Vec
# - or you can train your own model

# download a model
# 'glove-wiki-gigaword-300' (376.1 MB)
# 'word2vec-ruscorpora-300' (198.8 MB)
# 'word2vec-google-news-300' (1.6 GB)
# vectorizer = gensim.downloader.load('word2vec-ruscorpora-300')

# train your own model
vectorizer = Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=1, workers=4).wv

In [30]:
# similar words to 'nlp'
#vectorizer.most_similar('nlp', topn=5)

In [31]:
def transform_text2(vectorizer, text, verbose=False):
    '''
    Transform the text in a vector[Word2Vec]
    vectorizer: sklearn.vectorizer
    text: str
    '''
    tokens = preprocess_string(text)
    words = [vectorizer[w] for w in tokens if w in vectorizer]
    if verbose:
        print('Text:', text)
        print('Vector:', [w for w in tokens if w in vectorizer])
    elif len(words):
        return np.mean(words, axis=0)
    else:
        return np.zeros((300), dtype=np.float32)

In [32]:
# just testing our Word2Vec
transform_text2(vectorizer, question, verbose=True)

Text: What is global warming?
Vector: ['global', 'warm']


In [33]:
# let's train the model to retrieve the document id 'c_id'
retriever2 = NearestNeighbors(**retriever_configs)

# vectorizer the documents, fit the retriever
X = documents['context'].apply(lambda x: transform_text2(vectorizer, x)).tolist()
retriever2.fit(X, documents['c_id'])

NearestNeighbors(metric='cosine', n_neighbors=10)